In [23]:
!!git clone https://github.com/zykls/folktables.git


["fatal: destination path 'folktables' already exists and is not an empty directory."]

In [119]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [120]:
df = pd.read_csv('folktables/adult_reconstruction.csv')
df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,gender,native-country,income,occupation
0,20,40,0,0,Private,Bachelors,13,Married-civ-spouse,Wife,White,Female,United-States,49100,Tech-support
1,40,21,0,0,Private,Some-college,10,Divorced,Own-child,White,Male,United-States,11500,Craft-repair
2,10,17,0,0,Private,11th,7,Never-married,Own-child,White,Male,United-States,2600,Other-service
3,50,51,0,0,Private,HS-grad,9,Married-civ-spouse,Husband,Asian-Pac-Islander,Male,Cambodia,38997,Sales
4,38,28,0,0,Private,Bachelors,13,Never-married,Not-in-family,White,Male,?,41400,Exec-managerial


In [121]:
cat_features = ['workclass', 'education', 'marital-status', 'relationship', 'race', 'gender', 'native-country', 'occupation']

encoded_df = df.copy()  # Use .copy() to avoid the warning
label_encoder = LabelEncoder()

for i in cat_features:
  encoded_df.loc[:, i] = label_encoder.fit_transform(encoded_df[i])

features = list(df.columns)
features.remove('income')

train_set, test_set = train_test_split(encoded_df, test_size=0.2)

X_train = np.array(train_set[features])
y_train = np.array(train_set['income'])

X_test = np.array(test_set[features])
y_test = np.array(test_set['income'])


print('Size train set:', train_set.shape[0])
print('Size test set:',test_set.shape[0])

Size train set: 39624
Size test set: 9907


In [122]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [123]:
test_df = test_set.copy()
test_df['pred_income'] = y_pred
test_df.head()

,hours-per-week,age,capital-gain,capital-loss,workclass,education,education-num,marital-status,relationship,race,gender,native-country,income,occupation,pred_income
29191,52,45,0,0,4,9,13,2,0,4,1,39,99999,12,56823.026857
9125,35,31,5178,0,4,11,9,2,5,4,0,39,61238,14,19180.070904
36457,40,34,0,0,4,8,11,2,0,4,1,39,24033,12,40740.151784
30108,8,17,0,0,4,0,6,4,3,4,1,39,1144,8,-4954.560065
46632,40,33,0,0,4,11,9,0,3,4,0,39,16900,8,25448.678247


In [124]:
def get_sp_reg_matrix(data, attribute):

  features = data[attribute].unique()

  matrix = pd.DataFrame(0.0, index=features, columns=features)

  for i in features:
    for j in features:
      matrix.loc[i, j] = data[data[attribute]==i]['pred_income'].mean() - data[data[attribute]==j]['pred_income'].mean()

  return matrix

race_sp = get_sp_reg_matrix(test_df, 'race')
gender_sp = get_sp_reg_matrix(test_df, 'gender')
native_country_sp = get_sp_reg_matrix(test_df, 'native-country')
marital_status_sp = get_sp_reg_matrix(test_df,'marital-status')


In [125]:
race_sp

,4,2,1,3,0
4,0.000000,9300.046720,-103.555537,10076.224504,9711.592124
2,-9300.046720,0.000000,-9403.602257,776.177784,411.545404
1,103.555537,9403.602257,0.000000,10179.780041,9815.147661
3,-10076.224504,-776.177784,-10179.780041,0.000000,-364.632380
0,-9711.592124,-411.545404,-9815.147661,364.632380,0.000000


In [126]:
gender_sp

,1,0
1,0.000000,15483.560893
0,-15483.560893,0.000000


In [127]:
native_country_sp

,39,33,11,2,24,10,40,13,0,26,...,35,36,37,7,25,14,20,1,18,16
39,0.000000,11221.580719,-693.569311,-3223.074996,134.613924,-4089.357556,11378.935074,18176.633558,-2584.823244,15937.438384,...,-4658.933732,-7489.722902,8763.695514,3734.694108,15600.419785,8385.275036,-3979.890212,5227.024197,-7716.379958,26938.570556
33,-11221.580719,0.000000,-11915.150029,-14444.655715,-11086.966795,-15310.938275,157.354355,6955.052839,-13806.403963,4715.857665,...,-15880.514451,-18711.303621,-2457.885205,-7486.886611,4378.839066,-2836.305683,-15201.470930,-5994.556521,-18937.960676,15716.989838
11,693.569311,11915.150029,0.000000,-2529.505685,828.183234,-3395.788246,12072.504385,18870.202868,-1891.253934,16631.007694,...,-3965.364422,-6796.153591,9457.264824,4428.263419,16293.989095,9078.844347,-3286.320901,5920.593508,-7022.810647,27632.139867
2,3223.074996,14444.655715,2529.505685,0.000000,3357.688920,-866.282560,14602.010070,21399.708554,638.251752,19160.513380,...,-1435.858736,-4266.647906,11986.770510,6957.769104,18823.494781,11608.350032,-756.815216,8450.099193,-4493.304962,30161.645552
24,-134.613924,11086.966795,-828.183234,-3357.688920,0.000000,-4223.971480,11244.321151,18042.019634,-2719.437168,15802.824460,...,-4793.547656,-7624.336826,8629.081590,3600.080184,15465.805861,8250.661112,-4114.504135,5092.410274,-7850.993881,26803.956633
10,4089.357556,15310.938275,3395.788246,866.282560,4223.971480,0.000000,15468.292631,22265.991114,1504.534312,20026.795940,...,-569.576176,-3400.365346,12853.053070,7824.051664,19689.777341,12474.632592,109.467345,9316.381754,-3627.022401,31027.928113
40,-11378.935074,-157.354355,-12072.504385,-14602.010070,-11244.321151,-15468.292631,0.000000,6797.698483,-13963.758319,4558.503310,...,-16037.868806,-18868.657976,-2615.239561,-7644.240966,4221.484711,-2993.660038,-15358.825286,-6151.910877,-19095.315032,15559.635482
13,-18176.633558,-6955.052839,-18870.202868,-21399.708554,-18042.019634,-22265.991114,-6797.698483,0.000000,-20761.456802,-2239.195174,...,-22835.567290,-25666.356460,-9412.938044,-14441.939450,-2576.213773,-9791.358522,-22156.523769,-12949.609360,-25893.013515,8761.936999
0,2584.823244,13806.403963,1891.253934,-638.251752,2719.437168,-1504.534312,13963.758319,20761.456802,0.000000,18522.261628,...,-2074.110488,-4904.899658,11348.518758,6319.517352,18185.243029,10970.098280,-1395.066967,7811.847442,-5131.556713,29523.393801
26,-15937.438384,-4715.857665,-16631.007694,-19160.513380,-15802.824460,-20026.795940,-4558.503310,2239.195174,-18522.261628,0.000000,...,-20596.372116,-23427.161286,-7173.742870,-12202.744276,-337.018599,-7552.163348,-19917.328596,-10710.414186,-23653.818341,11001.132173


In [128]:
marital_status_sp

,2,4,0,5,6,3,1
2,0.000000,21184.169660,4820.337408,22284.607955,18415.563954,14841.963252,-1764.148247
4,-21184.169660,0.000000,-16363.832252,1100.438296,-2768.605706,-6342.206408,-22948.317907
0,-4820.337408,16363.832252,0.000000,17464.270547,13595.226546,10021.625844,-6584.485655
5,-22284.607955,-1100.438296,-17464.270547,0.000000,-3869.044001,-7442.644704,-24048.756203
6,-18415.563954,2768.605706,-13595.226546,3869.044001,0.000000,-3573.600703,-20179.712201
3,-14841.963252,6342.206408,-10021.625844,7442.644704,3573.600703,0.000000,-16606.111499
1,1764.148247,22948.317907,6584.485655,24048.756203,20179.712201,16606.111499,0.000000
